In [1]:
!mkdir -p /tmp/pip/cache/
!cp /kaggle/input/sennet-hoa-pkgs/connected_components_3d-3.12.4-cp310-cp310-linux_x86_64.whl /tmp/pip/cache/
!ls /tmp/pip/cache/

connected_components_3d-3.12.4-cp310-cp310-linux_x86_64.whl


In [2]:
!pip install --no-index --find-links /tmp/pip/cache/ connected-components-3d

Looking in links: /tmp/pip/cache/
Processing /tmp/pip/cache/connected_components_3d-3.12.4-cp310-cp310-linux_x86_64.whl


In [ ]:
import cc3d

import sys, os
# for model, ckpt and helper code
sys.path.append('/kaggle/input/blood-vessel-segmentation-01')

from helper import *

import cv2
import pandas as pd
from glob import glob
import numpy as np
from skimage.filters import apply_hysteresis_threshold

from timeit import default_timer as timer
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt


In [4]:
# for cc3d.dust 
cfg = dotdict( 
    p_threshold=0.29, # best prediction probability threshold 
    cc_connectivity=6, # 6 best connected neighborhood  N.B. only 4,8 (2D) and 26, 18, and 6 (3D) are allowed
    cc_threshold=6552, # 6552 best  or options from 6474 - 6630 
    use_tta=True, 
)

In [5]:
data_dir = \
    '/kaggle/input/blood-vessel-segmentation' 
DATA_META = {
	'kidney_1_dense': dotdict(
		name='kidney_1_dense',
		image_no=(1000, 1000 + 1000),
		image_dir=f'{data_dir}/train'
	),
	'kidney_3_dense': dotdict(
		name='kidney_3_dense',
		image_no=(496, 996 + 1),
		image_dir=f'{data_dir}/train'
	),
	'kidney_5': dotdict(
		name='kidney_5',
		image_no=None,
		image_dir=f'{data_dir}/test'
	),
	'kidney_6': dotdict(
		name='kidney_6',
		image_no=None,
		image_dir=f'{data_dir}/test'
	),
}


mode = 'submit' #'local' 

if 'local' in mode:	
	valid_meta = [DATA_META['kidney_3_dense'], ]
if 'submit' in mode:
	valid_meta = [DATA_META['kidney_5'], DATA_META['kidney_6']]


## io input, etc function
def build_file_list(d):
	if d.image_no is not None:
		d.file = [f'{d.image_dir}/{d.name.replace("kidney_3_dense","kidney_3_sparse")}/images/{i:04d}.tif' for i in range(*d.image_no)]
	else:
		d.file = sorted(glob(f'{d.image_dir}/{d.name}/images/*.tif'))


for d in valid_meta:
	build_file_list(d)
print(valid_meta[0].name, '\n', valid_meta[0].file[:5]) 
print('MODE OK  !!!!')

kidney_5 
 ['/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0000.tif', '/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0001.tif', '/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0002.tif']
MODE OK  !!!!


In [6]:
def file_to_id(f):
	s = f.split('/')
	return s[-3] + '_' + s[-1][:-4]


def load_volume(d):
	volume = [
		cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in d.file
	]
	volume = np.stack(volume)
	return volume


def load_truth(d):
	truth = [
		cv2.imread(f.replace('/images/', '/labels/'), cv2.IMREAD_GRAYSCALE) for f in d.file
	]
	truth = np.stack(truth)
	truth = truth // 255
	return truth


def norm_by_percentile(x, low=10, high=99.8, alpha=0.01):
	xmin = np.percentile(x, low)
	xmax = np.percentile(x, high)
	x = (x - xmin) / (xmax - xmin)
	if 1:
		x[x > 1] = (x[x > 1] - 1) * alpha + 1
		x[x < 0] = (x[x < 0]) * alpha
	# x = np.clip(x,0,1)
	return x


def rle_encode(mask):
	pixel = mask.flatten()
	pixel = np.concatenate([[0], pixel, [0]])
	run = np.where(pixel[1:] != pixel[:-1])[0] + 1
	run[1::2] -= run[::2]
	rle = ' '.join(str(r) for r in run)
	if rle == '':
		rle = '1 0'
	return rle


def make_dummy_submission():
    segmentation_df = []
    for d in valid_meta:
        segmentation_df.append(
            pd.DataFrame(data={
                'id': [file_to_id(f) for f in d.file],
                'rle': ['1 0'] * len(d.file),
            })
        )
    segmentation_df = pd.concat(segmentation_df).reset_index(drop=True)
    segmentation_df.to_csv('segmentations.csv', index=False)
    return segmentation_df



print('DATASET OK  !!!!')

DATASET OK  !!!!


In [7]:
glob_file = glob(f'{data_dir}/test/kidney_5/images/*.tif')
segmentation_df = make_dummy_submission()
print(segmentation_df)

timm 0.9.16
<All keys matched successfully>
              id  rle
0  kidney_5_0000  1 0
1  kidney_5_0001  1 0
2  kidney_5_0002  1 0
3  kidney_6_0000  1 0
4  kidney_6_0001  1 0
5  kidney_6_0002  1 0
SUBMIT OK!!!


In [8]:
#debug
if (mode == 'local'):
    for d in valid_meta: 
        print(d.name)

        print('load_truth truth ...')
        truth = load_truth(d)
        prob  = np.load(f'prob.xyz{d.name}.npz')['prob']/255

        for th in [0.5, 0.4, 0.3, 0.2 ]:
            predict = (prob > th)

            hit, fp, t_sum, p_sum = np_hit_fp_metric(predict, truth)
            lb_score = fast_compute_surface_dice_score_from_tensor(predict, truth)

            print(checkpoint_file)
            print('th=',th)
            print('hit      :', hit / t_sum)
            print('fp       :', fp / p_sum)
            print('lb_score :', lb_score)
            print('')
            
            # post processing ---
            if cfg.cc_threshold > 0:
                predict = cc3d.dust(
                    predict,
                    connectivity=26,
                    threshold=cfg.cc_threshold,
                    in_place=False
                    )
                print('after cc3d.dust')
                hit, fp, t_sum, p_sum = np_hit_fp_metric(predict, truth)
                lb_score = fast_compute_surface_dice_score_from_tensor(predict, truth)
                print('hit      :', hit / t_sum)
                print('fp       :', fp / p_sum)
                print('lb_score :', lb_score)
                print('')
